In [1]:
import numpy as np
import pandas as pd

In [2]:
class rectifiedNewton():
    def __init__(self, A, b, c, x0):
        self.A = A
        self.b = b
        self.x = x0
        self.c = c
        print(self.x)
    def objGradient(self, x):
        g = self.A @ np.array(x) + self.b
        gNorm = np.linalg.norm(g, axis = 0)
        return np.array(g), gNorm
    def iterate(self, epsilon, t0 = 0.5, h0 = 0.1, alpha = 1.1, epsilon_LS = 0.001):
        '''
        params:
        epsilon:迭代终止界限，梯度范数小于epsilon
        t0:每次区间探索的初始值
        h0:每次区间探索的初始探索步长
        alpha:加步系数
        '''
        X, newtons, Norm, NormG, funValue = [], [], [], [], []
        g, gNorm = self.objGradient(self.x)
        newton = np.linalg.inv(self.A)@g
        newtonNorm = np.linalg.norm(newton)
        NormG.append(gNorm)
        Norm.append(newtonNorm)
        x = self.x
        X.append(x)
        funValue.append(self.fun(x))# 记录迭代函数值
        newtons.append(newton)
        while gNorm > epsilon:
            interval = self.searchInterval(t0, h0, alpha, x, newton)
            tk = self.goldenSplit(x, interval, epsilon_LS, newton)
            x = x - tk*newton
            funValue.append(self.fun(x))# 记录迭代函数值
            g, gNorm = self.objGradient(x)
            newton = np.linalg.inv(self.A)@g
            newtonNorm = np.linalg.norm(newton)
            NormG.append(gNorm)
            Norm.append(newtonNorm)
            X.append(x)
            newtons.append(newton)
        return X, newtons, Norm, NormG, funValue
    def fun(self, x):# 定义二次函数
        return (x.T)@self.A@x/2 + self.b@x + self.c
    def goldenSplit(self, x, t, epsilon, direction):
        '''
        黄金分割法
        param
        t:搜索区间
        direction:搜索方向
        return：搜索结果t
        '''
        a, b = t[0], t[1]
        t1 = a + 0.618*(b-a)
        t2 = a + 0.382*(b-a)
        fi1, fi2 = self.fun(x-t1*direction), self.fun(x-t2*direction)
        while abs(t2 - t1) >= epsilon:
            if fi1 < fi2:
                a = t2
                t2, fi2 = t1, fi1
                t1 = a + b - t2
                fi1 = self.fun(x-t1*direction)
            else:
                b = t1
                t1, fi1 = t2, fi2
                t2 = a + 0.382*(b-a)
                fi2 = self.fun(x-t2*direction)
        return (t1 + t2)/2
    def searchInterval(self, t0, h0, alpha, x, direction):
        '''
        确定搜索区间
        '''
        k, t, h= 0, t0, h0
        tk = t
        tk_1 = tk + h
        fik, fik_1 = self.fun(x-tk*direction), self.fun(x-tk_1*direction)
        if fik > fik_1:
            h = alpha*h
            t, tk = tk, tk_1
            fik = fik_1
            tk_1 = tk + h
            fik_1 = self.fun(x-tk_1*direction)
            while fik > fik_1:
                h = alpha*h
                t, tk = tk, tk_1
                fik = fik_1
                tk_1 = tk + h
                fik_1 = self.fun(x-tk_1*direction)
        else:
            h = -h
            t = tk_1
            tk_1 = tk + h
            fik_1 = self.fun(x-fik_1*direction)
            while fik > fik_1:
                h = alpha*h
                t, tk = tk, tk_1
                fik = fik_1
                tk_1 = tk + h
                fik_1 = self.fun(x-tk_1*direction)
        return [min([t, tk_1]), max([t, tk_1])]

In [3]:
A = np.array([[8, 0], [0, 4]])
b = np.array([9, -3])
c = 16
epsilon = 0.01
x0 = np.array([0, 0])
second = rectifiedNewton(A, b, c, x0)

[0 0]


In [4]:
X, newtons, Norm, NormG, funValue = second.iterate(0.01)
res3 = pd.DataFrame({'迭代x值':X, 'newton步长':newtons, 'newton步长范数':Norm, '梯度范数':NormG, '迭代函数值':funValue})

In [5]:
res3

,迭代x值,newton步长,newton步长范数,梯度范数,迭代函数值
0,"[0, 0]","[1.125, -0.75]",1.352082,9.486833,16.000000
1,"[-1.1246137798518623, 0.7497425199012415]","[0.000386220148137717, -0.000257480098758478]",0.000464,0.003257,9.812501
